1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [299]:
import numpy as np
from tqdm import tqdm

In [300]:
def calc_logloss(y, y_pred):
    y_pred = np.clip(y_pred, 1e-8, (1 - 1e-8))
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [301]:
# Пример применения 1
y1 = np.array([1, 0])
y_pred1 = np.array([0, 0.2])
calc_logloss(y1, y_pred1)

9.321912147633288

In [302]:
# Пример применения 2
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.2])
calc_logloss(y1, y_pred1)

0.11157178065710491

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [303]:
def standard_scale(x):
    return (x - x.mean()) / x.std()


def sigmoid(z):    
    return (1 / (1 + np.exp(-z)))

In [304]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [305]:
X_st = X.copy()
X_st[:, 1:4] = standard_scale(X[:, 1:4])

In [306]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z) 
        err = calc_logloss(y, y_pred)
           
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        
        if i % (iterations / 10) == 0:
            print(i, W, err)    
        
    return W, err

def grid_search(x, y):
    iterations = np.logspace(2, 4, 4, dtype=int)
    etas = np.linspace(1e-2, 5, 10)

    best_error = np.inf
    best_params = {}

    for iteration in tqdm(iterations):
        for eta in etas:
            W, error = eval_model(x, y, iterations=iteration, eta=eta)
            if error < best_error:
                best_error = error
                best_params = {'iteration' : iteration, 'eta': eta}
    return f'Меньшая ошибка {best_error} при параметрах {best_params}'

In [307]:
grid_search(X_st, y)

 25%|█████████████████████                                                               | 1/4 [00:00<00:00,  9.28it/s]

0 [ 0.49573451 -0.13768295  0.6458828   1.5236234 ] 0.7220371880948343
10 [ 0.48637599 -0.13213295  0.62843494  1.52929421] 0.7174375767336868
20 [ 0.47778124 -0.12704245  0.61205977  1.53450338] 0.7134428408857539
30 [ 0.46990467 -0.12238398  0.59670555  1.53927849] 0.7099810965698217
40 [ 0.46270176 -0.1181307   0.58231988  1.54364646] 0.7069871182775452
50 [ 0.45612931 -0.11425658  0.56885039  1.54763342] 0.7044021492847485
60 [ 0.45014566 -0.11073651  0.55624519  1.55126455] 0.7021736074554744
70 [ 0.44471083 -0.10754642  0.54445337  1.55456402] 0.7002547164789252
80 [ 0.43978666 -0.10466333  0.53342533  1.55755491] 0.6986040889674408
90 [ 0.43533687 -0.10206539  0.52311306  1.56025915] 0.6971852837414756
0 [ 0.44141902 -0.10545049  0.54576446  1.55653225] 0.7220371880948343
10 [ 0.42748043 -0.1012565   0.35355808  1.56586812] 0.6875738845556977
20 [ 0.46469442 -0.12748642  0.31022839  1.54439742] 0.686704569172651
30 [ 0.48592879 -0.14392092  0.28535256  1.53264413] 0.686413195993

 50%|██████████████████████████████████████████                                          | 2/4 [00:00<00:00,  4.48it/s]

 [ 0.33278803 -0.04098557  0.34552778  1.62234993] 0.7220371880948343
0 [ 0.27847254 -0.00875311  0.24540944  1.65525877] 0.7220371880948343
0 [0.22415704 0.02347935 0.1452911  1.68816762] 0.7220371880948343
0 [0.16984155 0.05571181 0.04517276 1.72107646] 0.7220371880948343
0 [ 0.11552605  0.08794427 -0.05494558  1.7539853 ] 0.7220371880948343
0 [ 0.06121056  0.12017673 -0.15506392  1.78689415] 0.7220371880948343
0 [ 0.00689506  0.15240919 -0.25518226  1.81980299] 0.7220371880948343
0 [ 0.49573451 -0.13768295  0.6458828   1.5236234 ] 0.7220371880948343
0 [ 0.44141902 -0.10545049  0.54576446  1.55653225] 0.7220371880948343
0 [ 0.38710353 -0.07321803  0.44564612  1.58944109] 0.7220371880948343
0 [ 0.33278803 -0.04098557  0.34552778  1.62234993] 0.7220371880948343
0 [ 0.27847254 -0.00875311  0.24540944  1.65525877] 0.7220371880948343
0 [0.22415704 0.02347935 0.1452911  1.68816762] 0.7220371880948343
0 [0.16984155 0.05571181 0.04517276 1.72107646] 0.7220371880948343
0 [ 0.11552605  0.08794

 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:01<00:00,  1.38it/s]

0 [ 0.00689506  0.15240919 -0.25518226  1.81980299] 0.7220371880948343
0 [ 0.49573451 -0.13768295  0.6458828   1.5236234 ] 0.7220371880948343
1000 [ 0.45343587 -0.11947099  0.32332412  1.55086528] 0.6868524028054025
2000 [ 0.49097046 -0.14853526  0.27926927  1.5300943 ] 0.6863550546470853
3000 [ 0.50459426 -0.16294408  0.26234434  1.52386305] 0.6862779857389033
4000 [ 0.50896377 -0.17168084  0.255862    1.52325908] 0.6862634326769661
5000 [ 0.50977618 -0.17823734  0.25338914  1.52481806] 0.6862580997051531
6000 [ 0.50922424 -0.18395748  0.25245357  1.52720667] 0.6862541224581441
7000 [ 0.5081494  -0.18935701  0.25210709  1.52991324] 0.6862503444295719
8000 [ 0.50687425 -0.19463364  0.25198632  1.53274161] 0.6862465957651229
9000 [ 0.50552238 -0.19986311  0.25195202  1.53561663] 0.6862428515306853
0 [ 0.44141902 -0.10545049  0.54576446  1.55653225] 0.7220371880948343
1000 [ 0.43912901 -0.44676837  0.25283918  1.67350456] 0.6860654062529616
2000 [ 0.36033542 -0.73983556  0.25393187  1.83

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.85s/it]


"Меньшая ошибка 0.6782743647686446 при параметрах {'iteration': 10000, 'eta': 2.227777777777778}"

In [308]:
W, err = eval_model(X_st, y, iterations=10000, eta=2.227777777777778)

0 [ 0.27847254 -0.00875311  0.24540944  1.65525877] 0.7220371880948343
1000 [ 0.20716094 -1.31021852  0.25605854  2.15636544] 0.6854458457584679
2000 [-0.10121696 -2.46100808  0.26034942  2.80121062] 0.6846220330186017
3000 [-0.40727779 -3.60655413  0.26462089  3.44462574] 0.6838050913167155
4000 [-0.71103277 -4.74688161  0.26887304  4.08661734] 0.6829949569532331
5000 [-1.0124931  -5.88201547  0.27310595  4.7271919 ] 0.6821915667012354
6000 [-1.31166995 -7.0119806   0.27731969  5.36635596] 0.6813948578073907
7000 [-1.60857451 -8.1368019   0.28151435  6.00411601] 0.6806047679927246
8000 [-1.90321793 -9.25650419  0.28569002  6.64047856] 0.6798212354532376
9000 [ -2.19561135 -10.37111232   0.28984676   7.27545011] 0.6790441988603728


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred_proba.

In [309]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(X, W))
    return y_pred_proba 

In [310]:
calc_pred_proba(W, X_st)

array([0.44363906, 0.47079281, 0.47595649, 0.43606277, 0.56597251,
       0.4897835 , 0.56581171, 0.60624423, 0.43689654, 0.5101378 ])

4. Создайте функцию calc_pred, возвращающую предсказанный класс. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred.

In [311]:
def calc_pred(W, X):
    y_pred_proba = calc_pred_proba(W, X)
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)  # 0.5 взяла условна        
    return y_pred

In [312]:
y_pred = calc_pred(W, X_st)
y_pred, y

(array([0, 0, 0, 0, 1, 0, 1, 1, 0, 1]),
 array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.]))

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [313]:
def accuracy_metric(y, y_pred):
    return np.mean(y == y_pred)


def confusion_matrix(y, y_pred):
    k = len(np.unique(y))
    conf_matrix = np.zeros((k, k))
    for i in range(len(y)):
            if y[i] == y_pred[i] == 1:
                conf_matrix[0][0] += 1 # TP
            elif y[i] == y_pred[i] == 0:
                conf_matrix[1][1] += 1 # TN
            elif y[i] != y_pred[i] and y[i] == 1: # FN
                conf_matrix[1][0] += 1
            elif y[i] != y_pred[i] and y[i] == 0: # FP
                conf_matrix[0][1] += 1
    return conf_matrix                

            
def precision_metric(y, y_pred):
    conf_matrix = confusion_matrix(y, y_pred)
    TP = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    return TP / (TP + FP)


def recall_metric(y, y_pred):
    conf_matrix = confusion_matrix(y, y_pred)
    TP = conf_matrix[0][0]
    FN = conf_matrix[1][0]
    return TP / (TP + FN)


def f1_score(precision, recall):
    return 2 * precision * recall / (precision + recall)


In [314]:
accuracy_metric(y, y_pred)

0.7

In [315]:
conf_matrix = confusion_matrix(y, y_pred)
conf_matrix

array([[3., 1.],
       [2., 4.]])

In [316]:
precision = precision_metric(y, y_pred)
precision

0.75

In [317]:
recall = recall_metric(y, y_pred)
recall

0.6

In [318]:
f1_score(precision, recall)

0.6666666666666665

6. Могла ли модель переобучиться? Почему?

Нет, метрики не высокие.